In [13]:
# @Author: Kyunghun Lee 
# Created: Wed Sep 20 08:44:56 EDT 2023
# Updated: Thu Sep 28 12:03:56 EDT 2023 (version2): Fixed missing entries (KL)

version = 2
import pandas as pd
import numpy as np

# Load the Excel file into a DataFrame
# inFile = 'input/24642_8.4.23_InPerson.csv'
df = pd.read_csv('input/24642_8.4.23_InPerson.csv')
sdan = df['SDAN'][0]
display(df)
display(sdan)

App Name  Version  Build   SDAN Section Start Time Section End Time  \
0      TAP-it        7      6  24642            02:20.6          02:36.7   
1      TAP-it        7      6  24642            02:36.7          02:39.3   
2      TAP-it        7      6  24642            02:39.3          02:45.7   
3      TAP-it        7      6  24642            02:45.7          02:47.4   
4      TAP-it        7      6  24642            02:47.4          02:49.2   
...       ...      ...    ...    ...                ...              ...   
1022   TAP-it        7      6  24642            26:10.1          26:13.8   
1023   TAP-it        7      6  24642            26:13.8          26:23.5   
1024   TAP-it        7      6  24642            26:23.5          26:25.8   
1025   TAP-it        7      6  24642            26:25.8          26:27.4   
1026   TAP-it        7      6  24642            26:27.4          26:29.9   

      Duration (ms)                          Event         Section  \
0             16110             Getting subject ID  GetSubjectInfo   
1              2543  Slider Instruction Displayed:          Slider   
2              8940            Slider View - tense          Slider   
3              1675            Slider View - angry          Slider   
4              1824          Slider View - annoyed          Slider   
...             ...                            ...             ...   
1022           5934            Slider View - tense          Slider   
1023           9676            Slider View - angry          Slider   
1024           2308          Slider View - annoyed          Slider   
1025           1590            Slider View - happy          Slider   
1026           2474            Slider View - tired          Slider   

      Section Index  Timing Index  User Response  Correct Answer Correctness  \
0                 0           NaN          24642             NaN         NaN   
1                 1           NaN  Mouse clicked             NaN         NaN   
2                 1           0.0             51             NaN         NaN   
3                 2           0.0              0             NaN         NaN   
4                 3           0.0             23             NaN         NaN   
...             ...           ...            ...             ...         ...   
1022           1020           0.0             31             NaN         NaN   
1023           1021           0.0              0             NaN         NaN   
1024           1022           0.0             22             NaN         NaN   
1025           1023           0.0             81             NaN         NaN   
1026           1024           0.0             47             NaN         NaN   

      Response Time (ms) TrialType DisplayedStim  
0                    NaN       NaN           NaN  
1                    NaN       NaN           NaN  
2                    NaN       NaN           NaN  
3                    NaN       NaN           NaN  
4                    NaN       NaN           NaN  
...                  ...       ...           ...  
1022                 NaN       NaN           NaN  
1023                 NaN       NaN           NaN  
1024                 NaN       NaN           NaN  
1025                 NaN       NaN           NaN  
1026                 NaN       NaN           NaN  

[1027 rows x 17 columns]

24642

In [14]:
# Extract VAS responses
dict_VAS = {}

# Filter rows where 'Event' contains 'Slider View'
filtered_df = df[df['Event'].str.contains('Slider View')]

# Extract the name and store the result in a dictionary
for _, row in filtered_df.iterrows():
    name = (row['Event'].split('Slider View - ')[1]).capitalize()
    if 'Pre_'+name not in dict_VAS:
        dict_VAS['Pre_'+name] = row['User Response']
    else:
        dict_VAS['Post_'+name] = row['User Response']
display(dict_VAS)

{'Pre_Tense': '51',
 'Pre_Angry': '0',
 'Pre_Annoyed': '23',
 'Pre_Happy': '62',
 'Pre_Tired': '43',
 'Post_Tense': '31',
 'Post_Angry': '0',
 'Post_Annoyed': '22',
 'Post_Happy': '81',
 'Post_Tired': '47'}

In [15]:
# Output header
header = ["SDAN",'Input File','Home vs InPerson','S1 vs S2',
          "AX Correct","AX Incorrect","AX Total","AX Correct RT","AX Incorrect RT",
          "AX CorrectRate","AX IncorrectRate","AY Correct","AY Incorrect","AY Total","AY Correct RT",
          "AY Incorrect RT","AY CorrectRate","AY IncorrectRate","BX Correct","BX Incorrect",
          "BX Total","BX Correct RT","BX Incorrect RT","BX CorrectRate","BX IncorrectRate",
          "BY Correct","BY Incorrect","BY Total","BY Correct RT","BY Incorrect RT","BY CorrectRate",
          "BY IncorrectRate","Post-Correct CorrectRate","Post-Incorrect CorrectRate","Post-Correct RT",
          "Post-Incorrect RT","Post-Correct Efficiency","Post-Incorrect Efficiency","BY % CORRECT < 60%",
          "EXCLUDE","Master Tracker Coding"]

In [16]:
import pandas as pd
import numpy as np

# Initialize the result dictionary
result = {}

# Get unique trial types, ignoring empty or NaN
trial_types = df['TrialType'].dropna().unique()
trial_types = [t for t in trial_types if t != '' and t in ['AX', 'AY', 'BX', 'BY']]
trial_types.sort()

In [17]:
# check if the "Event" column contains the string "Clear" and if the "Response Time (ms)" column is empty (NaN) for the current row.
# If both conditions are met, we copy the "Response Time (ms)" value from the previous row to the current row.
for i in range(1, len(df)):  # start from 1 to avoid IndexError on i-1
    if 'Clear' in df.loc[i, 'Event'] and pd.isna(df.loc[i, 'Response Time (ms)']):
        df.loc[i, 'Response Time (ms)'] = df.loc[i - 1, 'Response Time (ms)']

In [18]:
# filter rows based on the condition that the values in the 'Response Time (ms)' column 
# are less than 3 times the standard deviation (3std) or more than 3 times the standard deviation (3std) from the mean
mean_response_time = df['Response Time (ms)'].mean()
std_response_time = df['Response Time (ms)'].std()

# Setting lower and upper bounds
lower_bound = mean_response_time - 3 * std_response_time
upper_bound = mean_response_time + 3 * std_response_time

# Filtering rows based on the condition
df = df[(df['Response Time (ms)'] >= lower_bound) & (df['Response Time (ms)'] <= upper_bound)]

# Convert it to milliseconds
df['Response Time (ms)'] = df['Response Time (ms)'] / 1000

In [19]:
import pandas as pd
import numpy as np

# Initialize the result dictionary
result = {}

# Get unique trial types, ignoring empty or NaN
trial_types = df['TrialType'].dropna().unique()
trial_types = [t for t in trial_types if t != '' and t in ['AX', 'AY', 'BX', 'BY']]
trial_types.sort()

# Variables for calculating Post-Correct CorrectRate and Post-Incorrect CorrectRate across all trials
total_post_correct_trials = 0
correct_following_correct = 0

total_post_incorrect_trials = 0
correct_following_incorrect = 0

# Variables to keep track of Post-Correct RT and Post-Incorrect RT across all trials
post_correct_rt_sum = 0
post_correct_rt_count = 0
post_incorrect_rt_sum = 0
post_incorrect_rt_count = 0

# Loop through each trial type
for trial in trial_types:
    df_trial = df[(df['TrialType'] == trial) & df['Event'].str.contains('Probe', case=False, na=False)].copy()
    df_trial = df_trial[df_trial['Event'].str.contains('Block', case=False, na=False)].copy()
    df_trial = df_trial[df_trial['Event'].str.contains('Clear', case=False, na=False)].copy()
    
    # Filter out rows where Correctness is NaN or empty
    df_trial = df_trial[df_trial['Correctness'].notna() & (df_trial['Correctness'] != '')]
        
    # Add a column for the previous trial's Correctness
    df_trial['PrevCorrectness'] = df_trial['Correctness'].shift()
    
    # Calculate mean RT for 'Probe' events, excluding NaN or empty entries
    mean_rt = df_trial.loc[df_trial['Event'].str.contains('Probe', case=False, na=False), 'Response Time (ms)'].dropna().mean()
    
    total_count = len(df_trial)
    correct_count = len(df_trial[df_trial['Correctness'] == 'Correct'])
    incorrect_count = len(df_trial[df_trial['Correctness'] == 'Incorrect'])
    
    # Calculate CorrectRate and IncorrectRate
    correct_rate = (correct_count / total_count) if total_count > 0 else 1
    incorrect_rate = (incorrect_count / total_count) if total_count > 0 else 1
    
    # Calculate 'Post-Correct CorrectRate' and 'Post-Incorrect CorrectRate'
    post_correct_correctrate = df_trial[(df_trial['PrevCorrectness'] == 'Correct') & (df_trial['Correctness'] == 'Correct')].shape[0] / df_trial[df_trial['PrevCorrectness'] == 'Correct'].shape[0] if df_trial[df_trial['PrevCorrectness'] == 'Correct'].shape[0] > 0 else 1 #np.nan
    post_incorrect_correctrate = df_trial[(df_trial['PrevCorrectness'] == 'Incorrect') & (df_trial['Correctness'] == 'Correct')].shape[0] / df_trial[df_trial['PrevCorrectness'] == 'Incorrect'].shape[0] if df_trial[df_trial['PrevCorrectness'] == 'Incorrect'].shape[0] > 0 else 1 #np.nan
    
    # Filter the data for correct and incorrect responses
    correct_responses = df_trial[(df_trial['Correctness'] == 'Correct') & (df_trial['Response Time (ms)'] > 0)]
    incorrect_responses = df_trial[(df_trial['Correctness'] == 'Incorrect') & (df_trial['Response Time (ms)'] > 0)]
    
    # Calculate mean RT for 'Probe' events, excluding NaN or empty entries and values <= 0
    correct_mean_rt = correct_responses['Response Time (ms)'].mean() if not correct_responses.empty else np.nan
    incorrect_mean_rt = incorrect_responses['Response Time (ms)'].mean() if not incorrect_responses.empty else np.nan
   
    # Calculate Post-Correct RT and Post-Incorrect RT
    post_correct_rt = df_trial[(df_trial['PrevCorrectness'] == 'Correct') & (df_trial['Response Time (ms)'] > 0)]['Response Time (ms)'].mean() if df_trial[df_trial['PrevCorrectness'] == 'Correct'].shape[0] > 0 else np.nan
    post_incorrect_rt = df_trial[(df_trial['PrevCorrectness'] == 'Incorrect') & (df_trial['Response Time (ms)'] > 0)]['Response Time (ms)'].mean() if df_trial[df_trial['PrevCorrectness'] == 'Incorrect'].shape[0] > 0 else np.nan
    
    # Update the running totals for Post-Correct RT and Post-Incorrect RT
    post_correct_rt_sum += df_trial[
        (df_trial['PrevCorrectness'] == 'Correct') & (df_trial['Response Time (ms)'].notna()) & (df_trial['Response Time (ms)'] > 0)
]['Response Time (ms)'].sum()
    
    post_correct_rt_count += df_trial[
        (df_trial['PrevCorrectness'] == 'Correct') & (df_trial['Response Time (ms)'].notna()) & (df_trial['Response Time (ms)'] > 0)
    ]['Response Time (ms)'].count()
    
    post_incorrect_rt_sum += df_trial[
        (df_trial['PrevCorrectness'] == 'Incorrect') & (df_trial['Response Time (ms)'].notna()) & (df_trial['Response Time (ms)'] > 0)
    ]['Response Time (ms)'].sum()
    
    post_incorrect_rt_count += df_trial[
        (df_trial['PrevCorrectness'] == 'Incorrect') & (df_trial['Response Time (ms)'].notna()) & (df_trial['Response Time (ms)'] > 0)
    ]['Response Time (ms)'].count()
    
    result[trial] = {
        f"{trial} Correct": correct_count,
        f"{trial} Incorrect": incorrect_count,
        f"{trial} Total": total_count,
        f"{trial} Correct Mean RT": correct_mean_rt,
        f"{trial} Incorrect Mean RT": incorrect_mean_rt,
        f"{trial} CorrectRate": correct_rate,
        f"{trial} IncorrectRate": incorrect_rate,
        f"{trial} Mean RT": mean_rt,
        f"{trial} Post-Correct CorrectRate": post_correct_correctrate,
        f"{trial} Post-Incorrect CorrectRate": post_incorrect_correctrate
    }
    
    # Calculate Post-Correct CorrectRate and Post-Incorrect CorrectRate across all trials
    total_post_correct_trials += len(df_trial[df_trial['PrevCorrectness'] == 'Correct'])
    correct_following_correct += len(df_trial[(df_trial['PrevCorrectness'] == 'Correct') & (df_trial['Correctness'] == 'Correct')])

    total_post_incorrect_trials += len(df_trial[df_trial['PrevCorrectness'] == 'Incorrect'])
    correct_following_incorrect += len(df_trial[(df_trial['PrevCorrectness'] == 'Incorrect') & (df_trial['Correctness'] == 'Correct')])

# Calculate Post-Correct CorrectRate and Post-Incorrect CorrectRate across all trials
post_correct_correctrate = correct_following_correct / total_post_correct_trials if total_post_correct_trials > 0 else 1 # np.nan
post_incorrect_correctrate = correct_following_incorrect / total_post_incorrect_trials if total_post_incorrect_trials > 0 else 1 # np.nan

result['Post-Correct CorrectRate'] = post_correct_correctrate
result['Post-Incorrect CorrectRate'] = post_incorrect_correctrate


# Calculate Post-Correct RT and Post-Incorrect RT across all trials
post_correct_rt = post_correct_rt_sum / post_correct_rt_count if post_correct_rt_count > 0 else np.nan
post_incorrect_rt = post_incorrect_rt_sum / post_incorrect_rt_count if post_incorrect_rt_count > 0 else np.nan

# Update the result dictionary with these values
result['Post-Correct RT'] = post_correct_rt
result['Post-Incorrect RT'] = post_incorrect_rt

# Calculate Post-Correct Efficiency and Post-Incorrect Efficiency across all trials
result['Post-Correct Efficiency'] = result['Post-Correct CorrectRate'] / result['Post-Correct RT']
result['Post-Incorrect Efficiency'] = result['Post-Incorrect CorrectRate'] / result['Post-Incorrect RT']

print(result)

{'AX': {'AX Correct': 104, 'AX Incorrect': 1, 'AX Total': 105, 'AX Correct Mean RT': 0.6064707017884615, 'AX Incorrect Mean RT': 0.6503239869999999, 'AX CorrectRate': 0.9904761904761905, 'AX IncorrectRate': 0.009523809523809525, 'AX Mean RT': 0.6068883521238095, 'AX Post-Correct CorrectRate': 0.9902912621359223, 'AX Post-Incorrect CorrectRate': 1.0}, 'AY': {'AY Correct': 8, 'AY Incorrect': 7, 'AY Total': 15, 'AY Correct Mean RT': 0.76181825995, 'AY Incorrect Mean RT': 0.777168427157143, 'AY CorrectRate': 0.5333333333333333, 'AY IncorrectRate': 0.4666666666666667, 'AY Mean RT': 0.7689816713133333, 'AY Post-Correct CorrectRate': 0.5714285714285714, 'AY Post-Incorrect CorrectRate': 0.5714285714285714}, 'BX': {'BX Correct': 15, 'BX Incorrect': 0, 'BX Total': 15, 'BX Correct Mean RT': 0.6180900017466666, 'BX Incorrect Mean RT': nan, 'BX CorrectRate': 1.0, 'BX IncorrectRate': 0.0, 'BX Mean RT': 0.6180900017466666, 'BX Post-Correct CorrectRate': 1.0, 'BX Post-Incorrect CorrectRate': 1}, 'BY':

In [20]:
display(result)

{'AX': {'AX Correct': 104,
  'AX Incorrect': 1,
  'AX Total': 105,
  'AX Correct Mean RT': 0.6064707017884615,
  'AX Incorrect Mean RT': 0.6503239869999999,
  'AX CorrectRate': 0.9904761904761905,
  'AX IncorrectRate': 0.009523809523809525,
  'AX Mean RT': 0.6068883521238095,
  'AX Post-Correct CorrectRate': 0.9902912621359223,
  'AX Post-Incorrect CorrectRate': 1.0},
 'AY': {'AY Correct': 8,
  'AY Incorrect': 7,
  'AY Total': 15,
  'AY Correct Mean RT': 0.76181825995,
  'AY Incorrect Mean RT': 0.777168427157143,
  'AY CorrectRate': 0.5333333333333333,
  'AY IncorrectRate': 0.4666666666666667,
  'AY Mean RT': 0.7689816713133333,
  'AY Post-Correct CorrectRate': 0.5714285714285714,
  'AY Post-Incorrect CorrectRate': 0.5714285714285714},
 'BX': {'BX Correct': 15,
  'BX Incorrect': 0,
  'BX Total': 15,
  'BX Correct Mean RT': 0.6180900017466666,
  'BX Incorrect Mean RT': nan,
  'BX CorrectRate': 1.0,
  'BX IncorrectRate': 0.0,
  'BX Mean RT': 0.6180900017466666,
  'BX Post-Correct Correct

In [21]:
def flatten(d, parent_key='', sep='_'):
    items = {}
    for k, v in d.items():
#         new_key = parent_key + sep + k if parent_key else k
        new_key = k
        if isinstance(v, dict):
            items.update(flatten(v, new_key, sep=sep))
        else:
            items[new_key] = v
    return items

# Initialize nested dictionary
nested_dict = {}

# Call the function to flatten the dictionary
flattened_dict = flatten(result)

# Add ID
new_item = {'ID': sdan,"Input File":inFile,}
if 'athome' in inFile.lower():
    new_item['Home vs InPerson'] = "At Home"
elif 'inperson' in inFile.lower():
    new_item['Home vs InPerson'] = "In Person"
else:
    new_item['Home vs InPerson'] = ""

if 'S1' in inFile:
    new_item['S1 vs S2'] = "S1"
elif "S2" in inFile:
    new_item['S1 vs S2'] = "S2"
else:
    new_item['S1 vs S2'] = ""

flattened_dict = {**new_item, **flattened_dict,**dict_VAS}

In [22]:
display(flattened_dict)

{'ID': 24642,
 'Input File': 'input/24642_8.4.23_InPerson.csv',
 'Home vs InPerson': 'In Person',
 'S1 vs S2': '',
 'AX Correct': 104,
 'AX Incorrect': 1,
 'AX Total': 105,
 'AX Correct Mean RT': 0.6064707017884615,
 'AX Incorrect Mean RT': 0.6503239869999999,
 'AX CorrectRate': 0.9904761904761905,
 'AX IncorrectRate': 0.009523809523809525,
 'AX Mean RT': 0.6068883521238095,
 'AX Post-Correct CorrectRate': 0.9902912621359223,
 'AX Post-Incorrect CorrectRate': 1.0,
 'AY Correct': 8,
 'AY Incorrect': 7,
 'AY Total': 15,
 'AY Correct Mean RT': 0.76181825995,
 'AY Incorrect Mean RT': 0.777168427157143,
 'AY CorrectRate': 0.5333333333333333,
 'AY IncorrectRate': 0.4666666666666667,
 'AY Mean RT': 0.7689816713133333,
 'AY Post-Correct CorrectRate': 0.5714285714285714,
 'AY Post-Incorrect CorrectRate': 0.5714285714285714,
 'BX Correct': 15,
 'BX Incorrect': 0,
 'BX Total': 15,
 'BX Correct Mean RT': 0.6180900017466666,
 'BX Incorrect Mean RT': nan,
 'BX CorrectRate': 1.0,
 'BX IncorrectRate':

In [23]:
# Exclusion criteria        
if (flattened_dict['BY CorrectRate'] < 0.70):
    flattened_dict['BY % CORRECT < 70%'] = 'y'
    flattened_dict['EXCLUDE'] = 'y'
else:
    flattened_dict['BY % CORRECT < 70%'] = 'n'
    flattened_dict['EXCLUDE'] = 'n'
    # flattened_dict['Master Tracker Coding'] = 'Good'

In [24]:
from datetime import datetime
import os

series = pd.Series(flattened_dict)
df = series.to_frame().transpose()


# Get the current date and time
now = datetime.now()

# Format the current date and time as a string, e.g., "20230926_142530"
datetime_str = now.strftime('%Y%m%d_%H%M%S')

# Construct the filename using the date and time string
filename = f'axcpt_mobile_summary_version{version}_{datetime_str}.csv'

# Save the DataFrame to a CSV file with the constructed filename
df.to_csv(filename, index=False)

# Assume the directory you want to save in is the current working directory
directory = os.getcwd()

# Create the absolute path
absolute_file_path = os.path.abspath(os.path.join(directory, filename))

# Now absolute_file_path contains the absolute path to the file
print(absolute_file_path)

/Users/leek13/github/BehaviorProcessing/AX-CPT/mobile/axcpt_mobile_summary_version2_20240105_100426.csv
